In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

# Import TensorFlow
import tensorflow as tf
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

from scripts_ml.ann_utils import *

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6815197232976252594
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7989946059685773471
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5325328194759015332
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 24199030375
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8072049123608107144
physical_device_desc: "device: 0, name: Quadro P6000, pci bus id: 0000:00:05.0, compute capability: 6.1"
]


In [3]:
#importing data
preproc_folder = "enriched_time_seq"
datafolder = "../data/preproc_traintest/"+preproc_folder+'/'
prefix_time_seq = 'time_2018-02-20_p180_bg_'
valid_code = '_val_26000_2000_'
trainfile = '_traindata'
testfile = '_testdata'
postfix_time_seq_val = '_190824_1812'
postfix_time_seq = '_190824_1321'
preproc_folder = "enriched_time_seq"
datafolder = "../data/preproc_traintest/"+preproc_folder+'/'
indexfile = '_fold_indexes'
expname = preproc_folder+"_p180"

[X_train, y_train, feature_labels] = pd.read_pickle(datafolder+prefix_time_seq+trainfile+postfix_time_seq+'.pkl') 
[X_test, y_test, feature_labels] = pd.read_pickle(datafolder+prefix_time_seq+testfile+postfix_time_seq+'.pkl') 
[val_X_train, val_y_train, val_feature_labels] = pd.read_pickle(datafolder+prefix_time_seq+valid_code+trainfile+postfix_time_seq_val+'.pkl') 
[val_X_test, val_y_test, val_feature_labels] = pd.read_pickle(datafolder+prefix_time_seq+valid_code+testfile+postfix_time_seq_val+'.pkl') 
indexes = pd.read_pickle(datafolder+prefix_time_seq+valid_code+indexfile+postfix_time_seq_val+'.pkl')

In [4]:
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense
from tensorflow.keras.models import Sequential

In [5]:
#recombining folds for grid search

val_X_all = []
val_y_all = []
indexes_tuples = []

count=0
start_tr=0

for idx in indexes:
    val_X_all.append(list(val_X_train[idx[0]]))
    val_y_all.append(list(val_y_train[idx[0]]))
    if count==0:
        test_idx = np.array(range(0, len(idx[1])))
    else:
        test_idx+=len(idx[1])
    #val_X_all.append(list(val_X_test[test_idx]))
    #val_y_all.append(list(val_y_test[test_idx]))
    
    
    if count==0:
        start_tst = len(idx[0])
    else:
        start_tr+=add_to_tr
        start_tst=start_tr+len(idx[0])
        
    indexes_tuples.append((np.array(range(start_tr, start_tr+len(idx[0]))), 
                          np.array(range(start_tst, start_tst+len(idx[1])))))
    
    add_to_tr = len(idx[0])+len(idx[1])
    
    count+=1

In [6]:
X = np.array(val_X_all[1:]).reshape((26000,7,35))
#y = np.array([val_X_all[1]])
X.shape

(26000, 7, 35)

In [7]:
y = np.array(val_y_all[1:]).reshape((26000,7,1))
y.shape

(26000, 7, 1)

In [66]:
model = Sequential()
#model.add(SimpleRNN(X.shape[2], return_sequences=True))
#model.add(SimpleRNN(X.shape[2], return_sequences=True))
#model.add(SimpleRNN(X.shape[2], return_sequences=True))
model.add(LSTM(X.shape[2], return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
model.add(LSTM(X.shape[2], return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
model.add(LSTM(X.shape[2], return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
model.add(LSTM(X.shape[2], return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
#model.add(LSTM(X.shape[2], return_sequences=True))
#model.add(LSTM(X.shape[2], return_sequences=True))
#model.add(LSTM(X.shape[2], return_sequences=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


In [67]:
#model.build(X.shape)
#model.summary()

In [68]:
with tf.device("/device:GPU:0"):
    model.fit(X, y, epochs=1000, batch_size=26000, verbose=1)

Epoch 1/1000
26000/26000 [==============================] - 6s 223us/sample - loss: 0.6927 - acc: 0.4697
Epoch 2/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.6790 - acc: 0.9102
Epoch 3/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.6655 - acc: 0.9174
Epoch 4/1000
26000/26000 [==============================] - 0s 9us/sample - loss: 0.6493 - acc: 0.9183
Epoch 5/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.6294 - acc: 0.9186
Epoch 6/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.6048 - acc: 0.9188
Epoch 7/1000
26000/26000 [==============================] - 0s 7us/sample - loss: 0.5755 - acc: 0.9188
Epoch 8/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.5425 - acc: 0.9189
Epoch 9/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.5089 - acc: 0.9189
Epoch 10/1000
26000/26000 [==============================] - 0s 8us/sam

Epoch 80/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.2397 - acc: 0.9231
Epoch 81/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.2376 - acc: 0.9234
Epoch 82/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.2365 - acc: 0.9236
Epoch 83/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.2347 - acc: 0.9236
Epoch 84/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.2340 - acc: 0.9234
Epoch 85/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.2316 - acc: 0.9237
Epoch 86/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.2299 - acc: 0.9239
Epoch 87/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.2284 - acc: 0.9240
Epoch 88/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.2273 - acc: 0.9239
Epoch 89/1000
26000/26000 [==============================] - 0s 

Epoch 159/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1667 - acc: 0.9372
Epoch 160/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1671 - acc: 0.9367
Epoch 161/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1666 - acc: 0.9376
Epoch 162/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1664 - acc: 0.9367
Epoch 163/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1651 - acc: 0.9385
Epoch 164/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1662 - acc: 0.9370
Epoch 165/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1646 - acc: 0.9379
Epoch 166/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1625 - acc: 0.9384
Epoch 167/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1630 - acc: 0.9392
Epoch 168/1000
26000/26000 [===========================

26000/26000 [==============================] - 0s 8us/sample - loss: 0.1446 - acc: 0.9454
Epoch 238/1000
26000/26000 [==============================] - 0s 9us/sample - loss: 0.1444 - acc: 0.9459
Epoch 239/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1434 - acc: 0.9468
Epoch 240/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1448 - acc: 0.9455
Epoch 241/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1433 - acc: 0.9462
Epoch 242/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1447 - acc: 0.9457
Epoch 243/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1445 - acc: 0.9457
Epoch 244/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1443 - acc: 0.9453
Epoch 245/1000
26000/26000 [==============================] - 0s 9us/sample - loss: 0.1434 - acc: 0.9468
Epoch 246/1000
26000/26000 [==============================] - 0s 8us/s

Epoch 316/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1328 - acc: 0.9492
Epoch 317/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1321 - acc: 0.9503
Epoch 318/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1312 - acc: 0.9504
Epoch 319/1000
26000/26000 [==============================] - 0s 9us/sample - loss: 0.1322 - acc: 0.9498
Epoch 320/1000
26000/26000 [==============================] - 0s 9us/sample - loss: 0.1303 - acc: 0.9502
Epoch 321/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1301 - acc: 0.9512
Epoch 322/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1301 - acc: 0.9505
Epoch 323/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1312 - acc: 0.9507
Epoch 324/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1305 - acc: 0.9508
Epoch 325/1000
26000/26000 [===========================

...

Epoch 630/1000
26000/26000 [==============================] - 0s 9us/sample - loss: 0.1046 - acc: 0.9596
Epoch 631/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1055 - acc: 0.9590
Epoch 632/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1038 - acc: 0.9602
Epoch 633/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1040 - acc: 0.9595
Epoch 634/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1042 - acc: 0.9598
Epoch 635/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1057 - acc: 0.9591
Epoch 636/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1058 - acc: 0.9593
Epoch 637/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1065 - acc: 0.9587
Epoch 638/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1074 - acc: 0.9589
Epoch 639/1000
26000/26000 [===========================

26000/26000 [==============================] - 0s 8us/sample - loss: 0.1037 - acc: 0.9598
Epoch 709/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1029 - acc: 0.9601
Epoch 710/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1009 - acc: 0.9610
Epoch 711/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1024 - acc: 0.9605
Epoch 712/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1003 - acc: 0.9607
Epoch 713/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1016 - acc: 0.9602
Epoch 714/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1010 - acc: 0.9606
Epoch 715/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1000 - acc: 0.9613
Epoch 716/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.1009 - acc: 0.9609
Epoch 717/1000
26000/26000 [==============================] - 0s 8us/s

Epoch 787/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0972 - acc: 0.9630
Epoch 788/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0977 - acc: 0.9626
Epoch 789/1000
26000/26000 [==============================] - 0s 9us/sample - loss: 0.0975 - acc: 0.9621
Epoch 790/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0980 - acc: 0.9620
Epoch 791/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0987 - acc: 0.9621
Epoch 792/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0980 - acc: 0.9622
Epoch 793/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0981 - acc: 0.9616
Epoch 794/1000
26000/26000 [==============================] - 0s 9us/sample - loss: 0.0976 - acc: 0.9626
Epoch 795/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0982 - acc: 0.9619
Epoch 796/1000
26000/26000 [===========================

26000/26000 [==============================] - 0s 9us/sample - loss: 0.0954 - acc: 0.9625
Epoch 866/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0940 - acc: 0.9632
Epoch 867/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0950 - acc: 0.9635
Epoch 868/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0956 - acc: 0.9631
Epoch 869/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0957 - acc: 0.9633
Epoch 870/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0960 - acc: 0.9629
Epoch 871/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0963 - acc: 0.9625
Epoch 872/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0946 - acc: 0.9632
Epoch 873/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0950 - acc: 0.9629
Epoch 874/1000
26000/26000 [==============================] - 0s 8us/s

Epoch 944/1000
26000/26000 [==============================] - 0s 9us/sample - loss: 0.0931 - acc: 0.9641
Epoch 945/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0933 - acc: 0.9639
Epoch 946/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0922 - acc: 0.9641
Epoch 947/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0928 - acc: 0.9642
Epoch 948/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0932 - acc: 0.9637
Epoch 949/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0931 - acc: 0.9637
Epoch 950/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0929 - acc: 0.9641
Epoch 951/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0931 - acc: 0.9641
Epoch 952/1000
26000/26000 [==============================] - 0s 8us/sample - loss: 0.0931 - acc: 0.9641
Epoch 953/1000
26000/26000 [===========================

In [69]:
y_scores = model.predict(X_test.reshape(15540,1,35))

In [70]:
y_scores = y_scores.flatten()

In [71]:
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

fpr, tpr, thresholds = roc_curve(y_test, y_scores)
auc=roc_auc_score(y_test, y_scores)
print("AUC {:.3f}".format(auc))

AUC 0.749
